In [ ]:
#Setting up environment
%pip install --upgrade tensorflow
%pip install pillow
%pip install numpy
%pip install matplotlib


In [ ]:
#Imports
import sys
import os
import numpy as np
from PIL import Image 
import tensorflow as tf
from keras.layers import Conv2D, LeakyReLU, BatchNormalization, Lambda, Input
from keras import Model
from keras.optimizers import Adam

In [ ]:
def load_images(directory, size=(128, 64)):  # Ensure size matches the expected input shape
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            img = Image.open(os.path.join(directory, filename))
            img = img.convert('L')  # Convert to grayscale
            img = img.resize(size, Image.Resampling.LANCZOS)  # Resize the image
            img_array = np.array(img)
            img_array = np.expand_dims(img_array, axis=-1)  # Add channel dimension
            images.append(img_array)
    return np.array(images) / 255.0


In [ ]:
high_res_path = r"C:\Users\kimam\Desktop\Project\datasets\hr_images"
low_res_path = r"C:\Users\kimam\Desktop\Project\datasets\lr_images"

# Load high-resolution and low-resolution images
high_res_images = load_images(high_res_path, size=(512, 256))  # Adjust the size as needed
low_res_images = load_images(low_res_path, size=(128, 64))


In [ ]:
# Example to check and reshape the images
print("Original shape:", low_res_images.shape)

if low_res_images.shape[1:3] != (128, 64):
    # Reshape the images
    low_res_images = np.reshape(low_res_images, newshape=(-1, 128, 64, 1))
    print("Reshaped to:", low_res_images.shape)


In [ ]:
#Generator Model (BARCNN)

def pixel_shuffle(scale):
    return lambda x: tf.nn.depth_to_space(x, scale)

def build_generator():
    inputs = Input(shape=(128, 64, 1))  # Starting with 128x64x1 grayscale images

    # 128x64x1 to 128x64x256
    x = Conv2D(256, (3, 3), padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)

    # 128x64x256 to 256x128x64 (upsampling)
    x = Lambda(pixel_shuffle(2))(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 256x128x64 to 256x128x256
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 256x128x256 to 512x256x64 (upsampling)
    x = Lambda(pixel_shuffle(2))(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Final layer: Conv2D to get to the desired channel depth
    x = Conv2D(1, (3, 3), padding='same')(x)  # For grayscale images

    model = Model(inputs, x)
    return model

generator = build_generator ()

In [ ]:
def build_discriminator():
    inputs = Input(shape=(512, 256, 1))  # Starting with 512x256x1 grayscale images

    # 512x256x1 to 256x128x128
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)

    # 256x128x128 to 128x64x256
    x = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 128x64x256 to 64x32x512
    x = Conv2D(512, (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # 64x32x512 to 32x16x1024
    x = Conv2D(1024, (3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Final layer to reduce to a single feature map
    x = Conv2D(1, (3, 3), padding='same')(x)

    model = Model(inputs, x)
    return model


In [ ]:
# Build Generator and Discriminator Models
generator = build_generator()
discriminator = build_discriminator()

# Print the summary for the Generator
print("Generator Model Summary:")
generator.summary()

# Print the summary for the Discriminator
print("\nDiscriminator Model Summary:")
discriminator.summary()


In [ ]:
#Defining Loss Functions and Optimizers
from keras.optimizers import Adam

# Loss function for the discriminator
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Loss function for the generator
def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)

generator_optimizer = Adam(1e-4)
discriminator_optimizer = Adam(1e-4)


In [ ]:
#Training Loop
@tf.function
def train_step(low_res_images, high_res_images):
# Reshape images if necessary
    low_res_images = tf.reshape(low_res_images, [-1, 128, 64, 1])
    high_res_images = tf.reshape(high_res_images, [-1, 512, 256, 1])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(low_res_images, training=True)

        real_output = discriminator(high_res_images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [ ]:
epochs = 2  # Number of epochs
batch_size = 1  # Batch size

# Convert images to TensorFlow datasets for easier batching
high_res_dataset = tf.data.Dataset.from_tensor_slices(high_res_images).batch(batch_size)
low_res_dataset = tf.data.Dataset.from_tensor_slices(low_res_images).batch(batch_size)

In [ ]:
# Generate high-resolution images from low-resolution images
generated_images = generator.predict(low_res_images[:5])  # Example: Generating 5 images


In [ ]:
def psnr(target, ref):
    # Assume target and ref are TensorFlow tensors
    return tf.image.psnr(target, ref, max_val=1.0)
def ssim(target, ref):
    # Assume target and ref are TensorFlow tensors
    return tf.image.ssim(target, ref, max_val=1.0)


In [ ]:
# After generating high-resolution images using the generator
low_res_images_tensor = tf.convert_to_tensor(low_res_images, dtype=tf.float32)
generated_images = generator.predict(low_res_images_tensor)

# Convert high-resolution images to a tensor and cast to float32
real_images_tensor = tf.convert_to_tensor(high_res_images, dtype=tf.float32)

# Check and reshape images here if necessary
print("Generated images shape:", generated_images.shape)
print("High-resolution images shape:", high_res_images.shape)
if generated_images.shape != high_res_images.shape:
    # Reshape code here
    generated_images = tf.image.resize(generated_images, (high_res_images.shape[1], high_res_images.shape[2]))
    generated_images_tensor = tf.convert_to_tensor(generated_images, dtype=tf.float32)

# Calculate PSNR and SSIM
average_psnr = psnr(generated_images_tensor, real_images_tensor)
average_ssim = ssim(generated_images_tensor, real_images_tensor)

print(f"Average PSNR: {average_psnr.numpy()} dB")
print(f"Average SSIM: {average_ssim.numpy()}")


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image as PILImage

# Function to display and save images
def display_and_save_images(images, save_dir):
    for i, img_tensor in enumerate(images):
        # Convert the tensor to a NumPy array and squeeze
        img = img_tensor.numpy().squeeze()

        # Display the image
        #%plt.imshow(img, cmap='gray')
        #%plt.axis('off')
        #%plt.show()

        # Save the image
        save_path = os.path.join(save_dir, f'generated_image_{i}.png')
        PILImage.fromarray((img * 255).astype(np.uint8)).save(save_path)
        print(f"Image {i} saved to {save_path}")

# Directory to save the images
save_directory = r"C:\Users\kimam\Desktop\Project\datasets\generated_images"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Display and save the generated images
display_and_save_images(generated_images, save_directory)
